In [190]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
# 엑셀 처리
import xlsxwriter
# 이미지 바이트 처리
from io import BytesIO
import urllib.request as req

### ChromeDriver browser 수행 안하고 시작

In [191]:
chrome_options = Options()
chrome_options.add_argument("--headless")

In [192]:
browser = webdriver.Chrome('./webdriver/chrome/chromedriver.exe', options=chrome_options)

## 1. Browser 실행환경 설정

In [248]:
# 크롬 브라우저 실행
browser = webdriver.Chrome('./webdriver/chrome/chromedriver.exe')

# 크롬 브라우저 내부 대기
browser.implicitly_wait(3)

# 브라우저 사이즈 조절
## 크기 조정 희망시 : brwoser.set_window_szie(1920,1200)
browser.maximize_window()

## 2. Selenium을 통해 page 내 html source 불러오기

In [249]:
browser.get('http://prod.danawa.com/list/?cate=112758&15main_11_02')

# 제조사별 더 보기 클릭1
# Explicity Wait(Implicty Wait과 달리 3초 내에 element를 찾으면 바로 실행한다. 더 빠른 효과)
WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.btn_spec_view.btn_view_more'))).click()
# 삼성전자, Apple 버튼 클릭
WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchMaker702'))).click()
WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchMaker1452'))).click()
time.sleep(1)

# 해당 page html 불러오기
#print("page html 불러오기 ", browser.page_source)

## 3. BeautifulSoup으로 원하는 정보 스크레이핑

* 스크래이핑 할 페이지 수 정하기

In [250]:
# 현재 페이지
cur_page = 1

# 스크래이핑 페이지 수
target_crawl_num = 2

* 엑셀, 워크 시트 선언

In [251]:
workbook = xlsxwriter.Workbook("./web_scraping.xlsx")
worksheet = workbook.add_worksheet()
excel_row = 1

worksheet.set_column(0, 0, 50) # column width 설정 
worksheet.set_column(1, 1, 20) # column width 설정 
worksheet.set_default_row(30) # row height 설정

* 웹 스크래이핑

In [252]:
i = 1
while cur_page <= target_crawl_num:
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    # print(soup.prettify())

    item_list = soup.select("div.main_prodlist li.prod_item.prod_layer > div.prod_main_info")

    print("page : ", cur_page)
    for item in item_list:
        if not v.find('div', class_='ad_header'):
            print("인기순위 : {}위".format(i))
            item_name = item.select_one("p.prod_name > a").text.strip()
            if item.select_one("div.thumb_image > a.thumb_link >img")['src'] == "//img.danawa.com/new/noData/img/noImg_130.gif":
                item_img = item.select_one("div.thumb_image > a.thumb_link >img")['data-original']
                img_data = BytesIO(req.urlopen(item.select_one("div.thumb_image > a.thumb_link >img")['data-original']).read())
            elif item.select_one("div.thumb_image > a.thumb_link >img")['src']:
                item_img = item.select_one("div.thumb_image > a.thumb_link >img")['src']
                img_data = BytesIO(req.urlopen(item.select_one("div.thumb_image > a.thumb_link >img")['src']).read())
            else:
                printf("HTML 페이지 내용이 변경되었으니 확인 필요")
            item_price = item.select_one("p.price_sect > a > strong").text.strip()
            print(item_name, "\n", item_img, "\n", item_price)
            print()
            i+=1
            
            # 엑셀 처리 
            worksheet.write('A%s' %excel_row, item_name)
            worksheet.write('B%s' %excel_row, item_price)
            worksheet.insert_image('C%s' %excel_row, item_name, {'image_data':img_data})
            excel_row +=1
            
    print()        
    browser.save_screenshot(f"./page_check{cur_page}.png")

    ## 다음 페이지 이동
    cur_page += 1
    time.sleep(3)

    WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'div.number_wrap > a:nth-child({cur_page})'))).click()

    # BeautifulSoup 인스턴스 삭제
    #del soup


workbook.close()

page :  1
인기순위 : 1위
APPLE 2019 맥북프로16 MVVK2KH/A 
 http://img.danawa.com/prod_img/500000/439/911/img/9911439_1.jpg?shrink=160:160&_v=20200319131535 
 3,165,000

인기순위 : 2위
삼성전자 갤럭시북 이온 NT950XCR-G58A 
 http://img.danawa.com/prod_img/500000/647/102/img/10102647_1.jpg?shrink=160:160&_v=20200304145022 
 1,479,000

인기순위 : 3위
삼성전자 갤럭시북 플렉스 NT930QCT-A38A 
 http://img.danawa.com/prod_img/500000/411/077/img/10077411_1.jpg?shrink=160:160&_v=20200304144507 
 1,350,000

인기순위 : 4위
APPLE 2019 맥북프로16 MVVJ2KH/A 
 http://img.danawa.com/prod_img/500000/355/911/img/9911355_1.jpg?shrink=160:160&_v=20200319131535 
 2,698,000

인기순위 : 5위
삼성전자 노트북5 NT550EBZ-AD3AH 
 http://img.danawa.com/prod_img/500000/830/258/img/10258830_1.jpg?shrink=160:160&_v=20200327102222 
 615,000

인기순위 : 6위
삼성전자 갤럭시북 플렉스 NT950QCT-A58A 
 http://img.danawa.com/prod_img/500000/282/077/img/10077282_1.jpg?shrink=160:160&_v=20200304144527 
 1,584,000

인기순위 : 7위
삼성전자 갤럭시북 플렉스 NT950QCG-X58A 
 http://img.danawa.com/prod_img/500000/482/102/img/10